### Notebook to produce shapefiles of electric vehicle charging stations.
* Data is fetched using the Department of Energy's National Renewable Energy Lab's programmatic API.
* Data is fetched for station locations in Massachusetts, and saved in one shapefile.
* Data is then clipped to the Boston Region MPO area, and saved in a second shapefile.

#### Reference document
[Data dictionary for data returned by API calls](https://afdc.energy.gov/data_download/alt_fuel_stations_format)

In [ ]:
# Import required packages
import pandas as pd
import geopandas as gp

#### User input required: specify name of output shapefiles

In [ ]:
output_ma_shapefile_name = 'ev_charging_stations_ma_20221014.shp'
output_brmpo_shapefile_name = 'ev_charging_stations_brmpo_20221014.shp'

Note: The GeoJSON returned by the API call contains property names that exceed the 10-character limit
on shapefile attribute names.  
When the data is exported to shapefile format,
__these attribute names are trunaced to 10 characters__.

In [ ]:
api_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson'

In [ ]:
# Request URL parameters

In [ ]:
api_key = 'YOUR_API_KEY_GOES_HERE'

In [ ]:
# Station is available
status = 'E'

In [ ]:
# Station is accessible to the public
access = 'public'

In [ ]:
# Station supplies electric 'fuel'.
# Note: This field appears to be ignored. See below.
fuel_type_code = 'ELEC'

In [ ]:
# Station is located in Massachusetts
state = 'MA'

In [ ]:
# Return all matching records
limit =  'all'

In [ ]:
# Form request URL

In [ ]:
request_url = api_url + '?'
request_url += 'api_key=' + api_key + '&'
request_url += 'status=' + status + '&'
request_url += 'access=' + access + '&'
request_url += 'fuel_type_code=' + fuel_type_code + '&'
request_url += 'state=' + state + '&'
request_url += 'limit=' + limit

In [ ]:
gdf = gp.read_file(request_url)

In [ ]:
gdf.head()

In [ ]:
# The server logic backing the API appears to ignore the 'fuel_type_code' parameter.
# We have to filter for electric charging stations manually.
ev_stations_gdf = gdf[gdf['fuel_type_code'] == 'ELEC']

In [ ]:
len(ev_stations_gdf)

In [ ]:
ev_stations_gdf.head()

In [ ]:
# Project the geo-dataframe to Massachusetts State Plane NAD83 Meters (EPSG:26986)

In [ ]:
ev_stations_gdf_epsg26986 = ev_stations_gdf.to_crs("EPSG:26986")

In [ ]:
# Sanity check: plot the points
ev_stations_gdf_epsg26986.plot()

In [ ]:
# Export the geo-dataframe for all locations in MA in shapefile format

In [ ]:
ev_stations_gdf_epsg26986.to_file(output_ma_shapefile_name)

In [ ]:
# Clip the dataframe for all locations in MA to the BRMPO area
# First, get a gdf for the BRMPO area

In [ ]:
ctps_wfs_url = 'https://ctps.org/maploc/wfs?service=wfs&'
request2_url = ctps_wfs_url + 'request=getFeature&' 
request2_url += 'typename=postgis:ctps_brmpo_boundary_poly&'
request2_url += 'outputFormat=json'

In [ ]:
# Note that this data is already in EPSG:26986
brmpo_gdf = gp.read_file(request2_url)

In [ ]:
brmpo_ev_stations_gdf = gp.clip(ev_stations_gdf_epsg26986, brmpo_gdf, keep_geom_type=True)

In [ ]:
len(brmpo_ev_stations_gdf)

In [ ]:
brmpo_ev_stations_gdf.head()

In [ ]:
# Sanity check: plot the points
brmpo_ev_stations_gdf.plot()

In [ ]:
brmpo_ev_stations_gdf.to_file(output_brmpo_shapefile_name)